In [ ]:
from sklearn.metrics import mean_squared_error
def compute_mse(Z, Y):
    """
    Compute the mean squared error (MSE) between predictions and ground truth.

    Parameters:
        Z (ndarray or csr_matrix): Predicted probabilities (n_nodes x n_classes).
        Y (ndarray or csr_matrix): Ground truth labels (n_nodes x n_classes).

    Returns:
        float: Mean squared error.
    """
    if hasattr(Z, "toarray"):  # Check if Z is sparse
        Z = Z.toarray()
    if hasattr(Y, "toarray"):  # Check if Y is sparse
        Y = Y.toarray()

    mse = np.mean((Z - Y) ** 2)  # Element-wise difference squared and mean
    return mse


def softmax(X):
    """
    Compute the softmax of each row of the matrix X, supporting both sparse and dense inputs.

    Parameters:
        X (csr_matrix or ndarray): Input matrix (n_nodes x n_classes).

    Returns:
        csr_matrix or ndarray: Matrix with softmax applied row-wise.
    """
    if isinstance(X, csr_matrix):
        # Sparse matrix handling
        X_dense = X.toarray()  # Convert sparse matrix to dense
        exp_X = np.exp(X_dense - np.max(X_dense, axis=1, keepdims=True))  # Stability
        row_sums = np.sum(exp_X, axis=1, keepdims=True)
        softmax_dense = exp_X / row_sums
        return csr_matrix(softmax_dense)  # Convert back to sparse
    else:
        # Dense matrix handling
        exp_X = np.exp(X - np.max(X, axis=1, keepdims=True))  # Stability
        row_sums = np.sum(exp_X, axis=1, keepdims=True)
        return exp_X / row_sums



def appnp(adj_matrix, H, alpha=0.1, K=10):
    """
    Implement the APPNP algorithm.

    Parameters:
        adj_matrix (ndarray or csr_matrix): The adjacency matrix of the graph.
        H (ndarray): Initial node feature matrix (n_nodes x n_features).
        alpha (float): Teleport probability.
        K (int): Number of iterations.

    Returns:
        Z (ndarray): Final predictions after propagation.
    """
    # Normalize the adjacency matrix
    adj_matrix = csr_matrix(adj_matrix)  # Ensure sparse matrix
    normalized_adj = normalize_adjacency(adj_matrix)

    # Initialize Z^(0)
    Z = H.copy()

    # Iterative propagation
    for _ in range(K):
        Z = (1 - alpha) * normalized_adj @ Z + alpha * H

    # Apply softmax at the end
    Z = softmax(Z)
    return Z